In [1]:
# import libraries
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType


# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('json-producer')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
         .getOrCreate())
sc = spark.sparkContext

In [2]:
mySchema = StructType([
 StructField("id", IntegerType()),
 StructField("nome", StringType()),
 StructField("idade", IntegerType()),
 StructField("sexo", IntegerType()),
 StructField("peso", DoubleType()),
 StructField("altura", IntegerType()),
 StructField("bpm", DoubleType()),
 StructField("pressao", DoubleType()),
 StructField("respiracao", DoubleType()),
 StructField("temperatura", DoubleType()),
 StructField("glicemia", DoubleType()),
 StructField("saturacao_oxigenio", DoubleType()),
 StructField("estado_atividade", IntegerType()),
 StructField("dia_de_semana", IntegerType()),
 StructField("periodo_do_dia", IntegerType()),
 StructField("semana_do_mes", IntegerType()),
 StructField("estacao_do_ano", IntegerType()),
 StructField("passos", IntegerType()),
 StructField("calorias", DoubleType()),
 StructField("distancia", DoubleType()),
 StructField("tempo", DoubleType()),
 StructField("total_sleep_last_24", DoubleType()),
 StructField("deep_sleep_last_24", DoubleType()),
 StructField("light_sleep_last_24", DoubleType()),
 StructField("awake_last_24", DoubleType()),
 StructField("fumante", IntegerType()),
 StructField("genetica", IntegerType()),
 StructField("gestante", IntegerType()),
 StructField("frutas", IntegerType()),
 StructField("vegetais", IntegerType()),
 StructField("alcool", IntegerType()),
 StructField("doenca_coracao", IntegerType()),     
 StructField("avc", IntegerType()),
 StructField("colesterol_alto", IntegerType()), 
 StructField("exercicio", IntegerType()), 
 StructField("timestampstr", StringType()),
 StructField("timestamp_epoch", StringType())    
])

In [3]:

csv_path = "/home/jovyan/work/csv"
csv_topic = "csv-data"
kafka_server = "kafka:29092"

streamingDataFrame = spark.readStream.schema(mySchema).csv(csv_path).filter("id is not null")



In [4]:
display(streamingDataFrame)

DataFrame[id: int, nome: string, idade: int, sexo: int, peso: double, altura: int, bpm: double, pressao: double, respiracao: double, temperatura: double, glicemia: double, saturacao_oxigenio: double, estado_atividade: int, dia_de_semana: int, periodo_do_dia: int, semana_do_mes: int, estacao_do_ano: int, passos: int, calorias: double, distancia: double, tempo: double, total_sleep_last_24: double, deep_sleep_last_24: double, light_sleep_last_24: double, awake_last_24: double, fumante: int, genetica: int, gestante: int, frutas: int, vegetais: int, alcool: int, doenca_coracao: int, avc: int, colesterol_alto: int, exercicio: int, timestampstr: string, timestamp_epoch: string]

In [5]:

streamingDataFrame.selectExpr("CAST(id AS STRING) AS key", "to_json(struct(*)) AS value") \
  .writeStream \
  .format("kafka") \
  .option("topic", csv_topic) \
  .option("kafka.bootstrap.servers", kafka_server) \
  .option("checkpointLocation", csv_path) \
  .start()


In [6]:

streamingDataFrame.selectExpr("CAST(id AS STRING) AS key", "to_json(struct(*)) AS value") \
    .writeStream \
    .format("console") \
    .start()


In [7]:
# Check active streams
for s in spark.streams.active:
    print("ID:{} | NAME:{}".format(s.id, s.name))

ID:b3831e8e-116c-4358-836b-990791af024c | NAME:None
ID:9681dfa9-1154-4456-97a3-a7d53b1aeffc | NAME:None


In [69]:
spark.stop()